<a href="https://colab.research.google.com/github/Paco-clod/OpenData-Comp-tion-on-Kaggle/blob/main/OpenData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Mining , Data wrangling

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Charger le fichier train_labels.csv
train_labels_df  = pd.read_csv("/content/drive/MyDrive/dates_types_classification/train_labels.csv")

In [ ]:
# Dossier contenant les images d'entraînement
train_dir = '/content/drive/MyDrive/dates_types_classification/train/'

# Dimensions des images que nous allons utiliser pour le modèle
image_size = (224, 224)

# Créer un mappage des labels en indices numériques
label_map = {label: idx for idx, label in enumerate(train_labels_df['label'].unique())}

def load_data():
    images = []
    image_labels = []

    # Pour chaque ligne du fichier CSV
    for index, row in train_labels_df.iterrows():
        # Charger l'image depuis le dossier train
        img_path = os.path.join(train_dir, row['filename'])
        img = load_img(img_path, target_size=image_size)
        img_array = img_to_array(img)

        # Ajouter l'image à la liste des images
        images.append(img_array)

        # Ajouter le label associé à l'image à la liste des labels
        image_labels.append(label_map[row['label']])

    # Convertir les listes en tableaux numpy
    images = np.array(images)
    image_labels = np.array(image_labels)

    # Retourner les images et les labels
    return images, image_labels

# Charger les données d'entraînement
train_images, train_labels = load_data()

# Convertir les labels en one-hot encoding
train_labels = to_categorical(train_labels, num_classes=len(label_map))

# Afficher la taille des données pour s'assurer que tout est correct
print(f'Nombre d\'images d\'entraînement : {train_images.shape[0]}')
print(f'Nombre de classes : {len(label_map)}')

Nombre d'images d'entraînement : 432
Nombre de classes : 7


## Modelisation

In [ ]:
# Charger le modèle EfficientNetB0 pré-entraîné sans la couche de classification (top)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Ajouter une couche de GlobalAveragePooling
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)  # Dropout pour éviter le sur-apprentissage
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_map), activation='softmax')(x)

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Geler les poids du modèle de base pour qu'il ne soit pas mis à jour pendant l'entraînement
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Compiler le modèle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher l'architecture du modèle
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_14 (Rescaling)    (None, 224, 224, 3)          0         ['input_10[0][0]']            
                                                                                                  
 normalization_7 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_14[0][0]']        
 tion)                                                                                            
                                                                                                  
 rescaling_15 (Rescaling)    (None, 224, 224, 3)          0         ['normalization_7[0][0]'

In [ ]:
# Séparer une partie des données pour la validation
# Diviser les données en entraînement et validation
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

# Normaliser les images (les mettre dans une plage [0, 1])
from tensorflow.keras.applications.efficientnet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=32)
train_images = preprocess_input(train_images)
val_images = preprocess_input(val_images)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

# Réduction du taux d'apprentissage si la validation ne s'améliore pas
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

# Entraînement du modèle avec le callback
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[reduce_lr]  # Ajout du callback ici
)

Epoch 1/10
11/11 [==============================] - 72s 5s/step - loss: 1.1621 - accuracy: 0.5884 - val_loss: 2.1701 - val_accuracy: 0.2069 - lr: 0.0010
Epoch 2/10
11/11 [==============================] - 54s 5s/step - loss: 0.3903 - accuracy: 0.8841 - val_loss: 2.0794 - val_accuracy: 0.2069 - lr: 0.0010
Epoch 3/10
11/11 [==============================] - 56s 5s/step - loss: 0.5662 - accuracy: 0.8406 - val_loss: 2.0953 - val_accuracy: 0.2069 - lr: 0.0010
Epoch 4/10
11/11 [==============================] - 51s 5s/step - loss: 0.3490 - accuracy: 0.8957 - val_loss: 2.2038 - val_accuracy: 0.1724 - lr: 0.0010
Epoch 5/10
11/11 [==============================] - ETA: 0s - loss: 0.2759 - accuracy: 0.8957
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
11/11 [==============================] - 56s 5s/step - loss: 0.2759 - accuracy: 0.8957 - val_loss: 2.6449 - val_accuracy: 0.1264 - lr: 0.0010
Epoch 6/10
11/11 [==============================] - 58s 5s/step - loss: 0.26

###Prédictions sur les Données de Test

In [ ]:
# Dossier contenant les images de test
test_dir = "/content/drive/MyDrive/dates_types_classification/test/"

# Fonction pour charger les images de test
def load_test_images(test_dir, image_size=(224, 224)):
    test_images = []
    filenames = []

    # Charger toutes les images du dossier test
    for filename in os.listdir(test_dir):
        img_path = os.path.join(test_dir, filename)
        img = load_img(img_path, target_size=image_size)
        img_array = img_to_array(img)
        test_images.append(img_array)
        filenames.append(filename)

    # Convertir en numpy array et normaliser
    test_images = np.array(test_images) / 255.0

    return test_images, filenames

In [ ]:
# Charger les images de test
test_images, test_filenames = load_test_images(test_dir)
test_images = preprocess_input(test_images)
# Faire des prédictions sur les images de test
predictions = model.predict(test_images)

# Convertir les prédictions en labels
predicted_labels = np.argmax(predictions, axis=1)

# Convertir les labels prédits en leurs noms correspondants
label_map_inv = {v: k for k, v in label_map.items()}
predicted_label_names = [label_map_inv[label] for label in predicted_labels]

4/4 [==============================] - 6s 1s/step


In [ ]:
# Créer un DataFrame pour la soumission
submission_df = pd.DataFrame({
    'filename': test_filenames,
    'label': predicted_label_names
})

# Sauvegarder la soumission dans un fichier CSV
submission_df.to_csv('submission3.csv', index=False)